**This Notebook predicts whether the image is labeled as Animals, Human faces or vehicles.**




In [ ]:
#To clear the memory
import tensorflow as tf
tf.keras.backend.clear_session()

In [46]:
# Mounting collab to google collab
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
#Unzipping the directory.
!unzip "/content/drive/MyDrive/Classes.zip" -d "/content/dataset"


Streaming output truncated to the last 5000 lines.
  inflating: /content/dataset/Classes/validation/vehicles/vehicles_05501.jpg  
  inflating: /content/dataset/Classes/validation/vehicles/vehicles_05502.jpg  
  inflating: /content/dataset/Classes/validation/vehicles/vehicles_05503.jpg  
  inflating: /content/dataset/Classes/validation/vehicles/vehicles_05504.jpg  
  inflating: /content/dataset/Classes/validation/vehicles/vehicles_05505.jpg  
  inflating: /content/dataset/Classes/validation/vehicles/vehicles_05506.jpg  
  inflating: /content/dataset/Classes/validation/vehicles/vehicles_05507.jpg  
  inflating: /content/dataset/Classes/validation/vehicles/vehicles_05508.jpg  
  inflating: /content/dataset/Classes/validation/vehicles/vehicles_05509.jpg  
  inflating: /content/dataset/Classes/validation/vehicles/vehicles_05510.jpg  
  inflating: /content/dataset/Classes/validation/vehicles/vehicles_05511.jpg  
  inflating: /content/dataset/Classes/validation/vehicles/vehicles_05512.jpg  
 

In [35]:
#Xception Model.  Concept of Transfer learning. Used the pretrained weights of Imagenet. I also had the image used to
import tensorflow as tf
from tensorflow.keras import regularizers
from tensorflow import keras

# Check if GPUs are available
print(f"Num GPUs Available: {len(tf.config.experimental.list_physical_devices('GPU'))}")

strategy = tf.distribute.MirroredStrategy()
print(f"Number of devices: {strategy.num_replicas_in_sync}")

with strategy.scope():
    base_model = tf.keras.applications.Xception(
        input_shape=(224, 224, 3),  # Xception expects 299x299 by default, but we resize to 224x224
        weights="imagenet",
        include_top=False
    )

    base_model.trainable = False  # Freeze the base model layers

    # Added custom layers which are trainable
    x = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)
    x = tf.keras.layers.Dropout(0.5)(x)  # Regularization
    x = tf.keras.layers.Dense(512, activation="relu")(x)
    prediction = tf.keras.layers.Dense(3, activation="softmax")(x)

    model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction)

    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
        loss="categorical_crossentropy",
        metrics=['accuracy']
    )

model.summary()

Num GPUs Available: 1
Number of devices: 1


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 224, 224,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1        │ (None, 111, 111,  │        864 │ input_layer[0][0] │
│ (Conv2D)            │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1_bn     │ (None, 111, 111,  │        128 │ block1_conv1[0][… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv1_act    │ (None, 111, 111,  │          0 │ block1_conv1_bn[… │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2        │ (None, 109, 109,  │     18,432 │ block1_conv1_act… │
│ (Conv2D)            │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2_bn     │ (None, 109, 109,  │        256 │ block1_conv2[0][… │
│ (BatchNormalizatio… │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1_conv2_act    │ (None, 109, 109,  │          0 │ block1_conv2_bn[… │
│ (Activation)        │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv1     │ (None, 109, 109,  │      8,768 │ block1_conv2_act… │
│ (SeparableConv2D)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv1_bn  │ (None, 109, 109,  │        512 │ block2_sepconv1[… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2_act │ (None, 109, 109,  │          0 │ block2_sepconv1_… │
│ (Activation)        │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2     │ (None, 109, 109,  │     17,536 │ block2_sepconv2_… │
│ (SeparableConv2D)   │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_sepconv2_bn  │ (None, 109, 109,  │        512 │ block2_sepconv2[… │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 55, 55,    │      8,192 │ block1_conv2_act… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block2_pool         │ (None, 55, 55,    │          0 │ block2_sepconv2_… │
│ (MaxPooling2D)      │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 55, 55,    │        512 │ conv2d[0][0]      │
│ (BatchNormalizatio… │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 55, 55,    │          0 │ block2_pool[0][0… │
│                     │ 128)              │            │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block3_sepconv1_act │ (None, 55, 55,    │          0 │ add[0][0]       

 Total params: 21,912,107 (83.59 MB)

 Trainable params: 1,050,627 (4.01 MB)

 Non-trainable params: 20,861,480 (79.58 MB)

In [48]:
#Data splitting using flow_from_directory
import tensorflow as tf
from tensorflow.keras.applications.xception import preprocess_input

datagenerator_kwargs = dict(
    preprocessing_function=preprocess_input,
)

dataflow_kwargs = dict(
    target_size=(224, 224),  # Resize all images to 224x224
    batch_size=512,
    interpolation="bilinear"  # Image resizing interpolation method
)

valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
    **datagenerator_kwargs
)

valid_generator = valid_datagenerator.flow_from_directory(
    directory='/content/dataset/Classes/validation',
    shuffle=False,
    **dataflow_kwargs
)

# Training data generator (With Augmentations). It generates data on fly
train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=30,  # Wider rotation range
    vertical_flip=True,
    horizontal_flip=True,
    width_shift_range=0.1,  # Slightly larger shift range
    height_shift_range=0.1,
    shear_range=0.1,
    zoom_range=0.3,  # Increased zoom range
    brightness_range=[0.3, 1.7],  # Increased brightness range
    **datagenerator_kwargs
)

train_generator = train_datagenerator.flow_from_directory(
    directory='/content/dataset/Classes/train',  # Training dataset
    shuffle=True,
    **dataflow_kwargs
)


Found 60000 images belonging to 3 classes.
Found 167998 images belonging to 3 classes.


In [49]:
# Total number of data
train_class_counts = train_generator.class_indices
print(f"Training class indices: {train_class_counts}")

train_class_counts = {class_name: 0 for class_name in train_generator.class_indices.keys()}
for class_name, class_index in train_generator.class_indices.items():
    train_class_counts[class_name] = sum(train_generator.classes == class_index)

print(f"Training data class distribution: {train_class_counts}")

valid_class_counts = valid_generator.class_indices
print(f"Validation class indices: {valid_class_counts}")

valid_class_counts = {class_name: 0 for class_name in valid_generator.class_indices.keys()}
for class_name, class_index in valid_generator.class_indices.items():
    valid_class_counts[class_name] = sum(valid_generator.classes == class_index)

print(f"Validation data class distribution: {valid_class_counts}")


Training class indices: {'animals': 0, 'human_faces': 1, 'vehicles': 2}
Training data class distribution: {'animals': np.int64(25200), 'human_faces': np.int64(84000), 'vehicles': np.int64(58798)}
Validation class indices: {'animals': 0, 'human_faces': 1, 'vehicles': 2}
Validation data class distribution: {'animals': np.int64(9000), 'human_faces': np.int64(30000), 'vehicles': np.int64(21000)}


In [38]:
# Saving the callbacks i.e. best model
import os
from tensorflow.keras.callbacks import ModelCheckpoint

save_dir = '/content/drive/MyDrive/classes_models'
os.makedirs(save_dir, exist_ok=True)

checkpoint = ModelCheckpoint(
    os.path.join(save_dir, "best_model_epoch_{epoch:02d}.h5"),
    monitor="val_accuracy",
    save_best_only=True,
    verbose=1
)


In [39]:
#Early stopping 6 but of no use because trained for 1 epoch
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor="val_accuracy", patience=6
)

In [40]:
#We have stopped training for second epoch as we got 99.93% accuracy on my validation data
model.fit(
              train_generator,
              epochs=30,
              validation_data=valid_generator,
              callbacks=[checkpoint,early_stop]
              # class_weight=class_weights

          )

Epoch 1/30
329/329 ━━━━━━━━━━━━━━━━━━━━ 0s 9s/step - accuracy: 0.9630 - loss: 0.1027
Epoch 1: val_accuracy improved from -inf to 0.99928, saving model to /content/drive/MyDrive/classes_models/best_model_epoch_01.h5


329/329 ━━━━━━━━━━━━━━━━━━━━ 3153s 10s/step - accuracy: 0.9631 - loss: 0.1025 - val_accuracy: 0.9993 - val_loss: 0.0031
Epoch 2/30
  5/329 ━━━━━━━━━━━━━━━━━━━━ 45:35 8s/step - accuracy: 0.9969 - loss: 0.0126

KeyboardInterrupt: 

In [54]:
# Trained model loading and making predictions, creating the test.csv as per the requirement
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.xception import preprocess_input
import tensorflow as tf
import pandas as pd
import numpy as np
import os

test_dir = '/content/dataset/Classes/test'

test_images = [os.path.join(test_dir, f) for f in os.listdir(test_dir)
              if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
df_test = pd.DataFrame({'filename': test_images})

#Create ImageDataGenerator
test_datagen = ImageDataGenerator(
    preprocessing_function=preprocess_input  # Xception-specific scaling for test data
)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=df_test,
    x_col='filename',
    y_col=None,  # No labels
    target_size=(224, 224),
    batch_size=1,
    shuffle=False,
    class_mode=None
)

#Loading model and predicting
model = tf.keras.models.load_model('/content/drive/MyDrive/classes_models/best_model_epoch_01.h5')
predictions = model.predict(test_generator)
predicted_classes = np.argmax(predictions, axis=1)

#Mapping to class names as keras maps the classes alphabetically
#0-animals, 1-human_faces, 2-vehicles
class_names = ['animals','human_faces','vehicles']
predicted_labels = [class_names[idx] for idx in predicted_classes]

#results DataFrame
results = pd.DataFrame({
    'image': [os.path.basename(f) for f in test_images],
    'class': predicted_labels
})

# Saving to CSV
results.to_csv('predictions.csv', index=False)
print(f"Predictions saved for {len(results)} images")
# The csv file has random images. You can cross verify the images and labels almost match by manually looking.

Found 12002 validated image filenames.


12002/12002 ━━━━━━━━━━━━━━━━━━━━ 105s 8ms/step
Predictions saved for 12002 images
